# Phase 2: Market Data Demo

This notebook demonstrates the market data functionality from Phase 2 of the IBKR Gateway project.

**Prerequisites:**
- IBKR Gateway or TWS running on paper trading mode
- Default connection: `127.0.0.1:4002`
- **US Securities Snapshot** subscription (for snapshot quotes)
- **US Equity & Options Add-On Streaming** subscription (for streaming quotes)

## Features Demonstrated:
1. **Snapshot quotes** - One-time market data request
2. **Streaming quotes** - Continuous real-time updates
3. Batch quote retrieval
4. Historical bar data (daily, intraday)
5. Input normalization (bar sizes, durations)
6. Error handling

**Note:** Real-time quotes require active market hours. Outside trading hours, quotes may timeout.

In [ ]:
# Add parent directory to path for imports
import sys
sys.path.insert(0, '..')

from ibkr_core import (
    IBKRClient,
    SymbolSpec,
    Quote,
    Bar,
    # Snapshot quotes
    get_quote,
    get_quotes,
    # Streaming quotes
    QuoteMode,
    StreamingQuote,
    get_streaming_quote,
    get_quote_with_mode,
    # Historical data
    get_historical_bars,
    # Normalization
    normalize_bar_size,
    normalize_duration,
    normalize_what_to_show,
    # Exceptions
    MarketDataError,
    MarketDataPermissionError,
    MarketDataTimeoutError,
    NoMarketDataError,
)

## 1. Connect to IBKR Gateway

In [ ]:
# Create client and connect
client = IBKRClient(mode="paper")
client.connect(timeout=10)

print(f"Connected: {client.is_connected}")
print(f"Mode: {client.mode}")
print(f"Accounts: {client.managed_accounts}")

## 2. Input Normalization

The market data module provides normalization functions that convert user-friendly inputs to IBKR-compatible formats.

In [ ]:
# Bar size normalization examples
print("Bar Size Normalization:")
bar_sizes = ["1m", "5m", "15m", "1h", "1d", "1w"]
for bs in bar_sizes:
    normalized = normalize_bar_size(bs)
    print(f"  '{bs}' -> '{normalized}'")

In [ ]:
# Duration normalization examples
print("Duration Normalization:")
durations = ["1d", "5d", "1w", "2w", "1mo", "3mo", "1y"]
for d in durations:
    normalized = normalize_duration(d)
    print(f"  '{d}' -> '{normalized}'")

In [ ]:
# What to show normalization
print("What To Show Normalization:")
data_types = ["trades", "MIDPOINT", "Bid", "ask"]
for dt in data_types:
    normalized = normalize_what_to_show(dt)
    print(f"  '{dt}' -> '{normalized}'")

## 3. Snapshot Quotes

Get current market data snapshots for instruments.

**Note:** Real-time quote data may require market data subscriptions. Paper accounts may receive delayed data or permission errors.

In [ ]:
# Get quote for AAPL
try:
    aapl_spec = SymbolSpec(symbol="AAPL", securityType="STK")
    quote = get_quote(aapl_spec, client, timeout_s=10.0)
    
    print("AAPL Quote:")
    print(f"  Symbol: {quote.symbol}")
    print(f"  ConId: {quote.conId}")
    print(f"  Bid: {quote.bid}")
    print(f"  Ask: {quote.ask}")
    print(f"  Last: {quote.last}")
    print(f"  Bid Size: {quote.bidSize}")
    print(f"  Ask Size: {quote.askSize}")
    print(f"  Volume: {quote.volume}")
    print(f"  Timestamp: {quote.timestamp}")
    print(f"  Source: {quote.source}")
    
except MarketDataPermissionError as e:
    print(f"Permission error (no market data subscription): {e}")
except MarketDataTimeoutError as e:
    print(f"Timeout: {e}")

## 4. Batch Quote Retrieval

Efficiently fetch quotes for multiple instruments in a single request.

In [ ]:
# Get quotes for multiple symbols
specs = [
    SymbolSpec(symbol="AAPL", securityType="STK"),
    SymbolSpec(symbol="MSFT", securityType="STK"),
    SymbolSpec(symbol="GOOGL", securityType="STK"),
]

try:
    quotes = get_quotes(specs, client, timeout_s=15.0)
    
    print("Batch Quotes:")
    print(f"{'Symbol':<10} {'Bid':>10} {'Ask':>10} {'Last':>10} {'Volume':>12}")
    print("-" * 55)
    for q in quotes:
        print(f"{q.symbol:<10} {q.bid:>10.2f} {q.ask:>10.2f} {q.last:>10.2f} {q.volume:>12.0f}")
        
except MarketDataError as e:
    print(f"Market data error: {e}")

## 5. Streaming Quotes (Real-Time)

Streaming quotes provide continuous real-time market data updates. This requires the **US Equity & Options Add-On Streaming** subscription.

**Key Differences from Snapshot:**
- Snapshot (`snapshot=True`): One-time request, returns immediately with current data
- Streaming (`snapshot=False`): Continuous subscription, updates as market moves

**Note:** Streaming quotes only work during market hours. Outside trading hours, you'll get timeout errors.

In [ ]:
# Basic streaming quote - context manager pattern (recommended)
aapl_spec = SymbolSpec(symbol="AAPL", securityType="STK")

try:
    with StreamingQuote(aapl_spec, client, timeout_s=15.0) as stream:
        print(f"Streaming active: {stream.is_active}")
        
        # Get current quote from stream
        quote = stream.get_current()
        print(f"\nAAP Streaming Quote:")
        print(f"  Bid: {quote.bid}")
        print(f"  Ask: {quote.ask}")
        print(f"  Last: {quote.last}")
        print(f"  Source: {quote.source}")  # Should be "IBKR_STREAMING"
        
except MarketDataTimeoutError as e:
    print(f"Timeout (markets may be closed): {e}")
except MarketDataPermissionError as e:
    print(f"Permission error: {e}")

In [ ]:
# Streaming quote updates - get continuous price updates
import time

aapl_spec = SymbolSpec(symbol="AAPL", securityType="STK")

try:
    with StreamingQuote(aapl_spec, client, timeout_s=15.0) as stream:
        print("Receiving streaming updates (max 5 or 3 seconds)...")
        print(f"{'Time':<12} {'Bid':>10} {'Ask':>10} {'Last':>10}")
        print("-" * 45)
        
        # Get updates - stops after 5 updates OR 3 seconds
        for quote in stream.updates(max_updates=5, duration_s=3.0, poll_interval_s=0.2):
            time_str = quote.timestamp.strftime("%H:%M:%S.%f")[:12]
            print(f"{time_str:<12} {quote.bid:>10.2f} {quote.ask:>10.2f} {quote.last:>10.2f}")
            
except MarketDataTimeoutError as e:
    print(f"Timeout (markets may be closed): {e}")
except MarketDataPermissionError as e:
    print(f"Permission error: {e}")

In [ ]:
# Compare Snapshot vs Streaming using get_quote_with_mode()
aapl_spec = SymbolSpec(symbol="AAPL", securityType="STK")

print("Comparing Snapshot vs Streaming modes:\n")

# Snapshot mode
try:
    snapshot_quote = get_quote_with_mode(aapl_spec, client, mode=QuoteMode.SNAPSHOT, timeout_s=10.0)
    print(f"SNAPSHOT: bid={snapshot_quote.bid:.2f}, ask={snapshot_quote.ask:.2f}, source={snapshot_quote.source}")
except MarketDataTimeoutError:
    print("SNAPSHOT: Timeout (markets may be closed)")

# Streaming mode (starts stream, gets one quote, stops)
try:
    streaming_quote = get_quote_with_mode(aapl_spec, client, mode=QuoteMode.STREAMING, timeout_s=10.0)
    print(f"STREAMING: bid={streaming_quote.bid:.2f}, ask={streaming_quote.ask:.2f}, source={streaming_quote.source}")
except MarketDataTimeoutError:
    print("STREAMING: Timeout (markets may be closed)")

## 6. Historical Bars - Daily Data

Fetch historical OHLCV data for analysis.

In [ ]:
# Get daily bars for AAPL
aapl_spec = SymbolSpec(symbol="AAPL", securityType="STK")

try:
    bars = get_historical_bars(
        aapl_spec,
        client,
        bar_size="1d",      # Daily bars
        duration="1mo",     # Last month
        what_to_show="TRADES",
        rth_only=True,      # Regular trading hours only
        timeout_s=30.0
    )
    
    print(f"AAPL Daily Bars ({len(bars)} bars):")
    print(f"{'Date':<12} {'Open':>10} {'High':>10} {'Low':>10} {'Close':>10} {'Volume':>12}")
    print("-" * 70)
    for bar in bars[-10:]:  # Show last 10
        date_str = bar.time.strftime("%Y-%m-%d")
        print(f"{date_str:<12} {bar.open:>10.2f} {bar.high:>10.2f} {bar.low:>10.2f} {bar.close:>10.2f} {bar.volume:>12.0f}")
        
except NoMarketDataError as e:
    print(f"No data available: {e}")
except MarketDataPermissionError as e:
    print(f"Permission error: {e}")

In [ ]:
# Visualize the data (if matplotlib is available)
try:
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    
    if 'bars' in dir() and bars:
        dates = [bar.time for bar in bars]
        closes = [bar.close for bar in bars]
        
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(dates, closes, 'b-', linewidth=1.5)
        ax.set_title(f'AAPL Daily Close Price (Last {len(bars)} Days)')
        ax.set_xlabel('Date')
        ax.set_ylabel('Price ($)')
        ax.grid(True, alpha=0.3)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("No bar data to plot")
except ImportError:
    print("matplotlib not installed - skipping visualization")

## 7. Historical Bars - Intraday Data

In [ ]:
# Get 5-minute bars for SPY
spy_spec = SymbolSpec(symbol="SPY", securityType="ETF")

try:
    intraday_bars = get_historical_bars(
        spy_spec,
        client,
        bar_size="5m",      # 5-minute bars
        duration="1d",      # Last day
        what_to_show="TRADES",
        rth_only=True,
        timeout_s=30.0
    )
    
    print(f"SPY 5-Minute Bars ({len(intraday_bars)} bars):")
    print(f"{'Time':<20} {'Open':>10} {'High':>10} {'Low':>10} {'Close':>10}")
    print("-" * 65)
    for bar in intraday_bars[-15:]:  # Show last 15
        time_str = bar.time.strftime("%Y-%m-%d %H:%M")
        print(f"{time_str:<20} {bar.open:>10.2f} {bar.high:>10.2f} {bar.low:>10.2f} {bar.close:>10.2f}")
        
except NoMarketDataError as e:
    print(f"No data available: {e}")
except MarketDataPermissionError as e:
    print(f"Permission error: {e}")

## 8. Futures Historical Data

In [ ]:
# Get daily bars for MES micro futures
mes_spec = SymbolSpec(symbol="MES", securityType="FUT")

try:
    mes_bars = get_historical_bars(
        mes_spec,
        client,
        bar_size="1d",
        duration="1w",      # Last week
        what_to_show="TRADES",
        rth_only=False,     # Include extended hours for futures
        timeout_s=30.0
    )
    
    print(f"MES Futures Daily Bars ({len(mes_bars)} bars):")
    print(f"{'Date':<12} {'Open':>10} {'High':>10} {'Low':>10} {'Close':>10}")
    print("-" * 55)
    for bar in mes_bars:
        date_str = bar.time.strftime("%Y-%m-%d")
        print(f"{date_str:<12} {bar.open:>10.2f} {bar.high:>10.2f} {bar.low:>10.2f} {bar.close:>10.2f}")
        
except NoMarketDataError as e:
    print(f"No data available: {e}")
except MarketDataPermissionError as e:
    print(f"Permission error: {e}")

## 9. Error Handling Examples

In [ ]:
# Invalid bar size
try:
    normalize_bar_size("invalid")
except ValueError as e:
    print(f"ValueError: {e}")

In [ ]:
# Invalid duration
try:
    normalize_duration("xyz")
except ValueError as e:
    print(f"ValueError: {e}")

In [ ]:
# Invalid symbol
try:
    invalid_spec = SymbolSpec(symbol="INVALIDXYZ", securityType="STK")
    get_historical_bars(invalid_spec, client, bar_size="1d", duration="5d")
except Exception as e:
    print(f"{type(e).__name__}: {e}")

## 10. Quote and Bar Model Inspection

In [ ]:
# Inspect Quote model fields
from pydantic import BaseModel
print("Quote Model Fields:")
for field_name, field_info in Quote.model_fields.items():
    print(f"  {field_name}: {field_info.annotation}")

In [ ]:
# Inspect Bar model fields
print("Bar Model Fields:")
for field_name, field_info in Bar.model_fields.items():
    print(f"  {field_name}: {field_info.annotation}")

In [ ]:
# JSON serialization example
if 'bars' in dir() and bars:
    sample_bar = bars[-1]
    print("Bar as JSON:")
    print(sample_bar.model_dump_json(indent=2))

## 11. Disconnect

In [ ]:
# Clean up - disconnect from gateway
client.disconnect()
print(f"Disconnected. Is connected: {client.is_connected}")

## Summary

Phase 2 provides comprehensive market data functionality with both snapshot and streaming support.

### Market Data Subscriptions Required:
- **US Securities Snapshot**: For `get_quote()` snapshot requests
- **US Equity & Options Add-On Streaming**: For `StreamingQuote` real-time updates

### Snapshot Quote Functions:
- `get_quote()`: Single instrument snapshot
- `get_quotes()`: Efficient batch snapshot retrieval
- `get_quote_with_mode(mode=QuoteMode.SNAPSHOT)`: Explicit mode selection

### Streaming Quote Functions:
- `StreamingQuote`: Context manager for continuous updates
- `get_streaming_quote()`: Factory function for StreamingQuote
- `get_quote_with_mode(mode=QuoteMode.STREAMING)`: Get one quote via streaming

### Historical Data:
- `get_historical_bars()`: Historical OHLCV bars (works even when markets closed)

### Input Normalization:
- `normalize_bar_size()`: Convert "1m", "5m", "1h", "1d" to IBKR format
- `normalize_duration()`: Convert "5d", "1w", "1mo" to IBKR format
- `normalize_what_to_show()`: Validate data types (TRADES, MIDPOINT, etc.)

### Exception Classes:
- `MarketDataError`: Base exception
- `MarketDataPermissionError`: No subscription for data
- `NoMarketDataError`: No data available for instrument
- `PacingViolationError`: IBKR rate limits exceeded
- `MarketDataTimeoutError`: Request timed out (common outside market hours)

### Key Features:
- Timeout-based polling with deadline enforcement
- Proper handling of NaN values from ib_insync
- Support for both date and datetime bar timestamps
- Integration with Phase 1 contract resolution
- Context manager pattern for clean resource management